### 200 Moving Day Average Crossover

##### Strategy: if there is a crossover between the 50 day moving average and the 200 day moving average, where the 50 day moving average is greater than the 200 day moving average, and there is high volumethen purchace $1000 worth of shares from that company. 

In [45]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import MarketOrderRequest, LimitOrderRequest
from alpaca.data.historical import StockHistoricalDataClient
from alpaca_trade_api.rest import REST


API_KEY = "PKNXRGRC2A6PAVABSRKP"
SECRET_KEY = "NjfSwiQ3lb431zYfyPRaCA2PfTAhySdMV6M1Ot0S"
trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)
data_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)
api = REST(API_KEY, SECRET_KEY, base_url='https://paper-api.alpaca.markets')


#### Get S&P 500

In [49]:
# Wikipedia URL containing the current S&P 500 companies list
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Load the table containing S&P 500 companies into a DataFrame
tables = pd.read_html(url)

# The table we want is usually the first one
sp500_table = tables[0]

# Extract the ticker symbols
sp500_stocks = sp500_table['Symbol']
positions = api.list_positions()


In [ ]:
#return whether or not stock is present in portfolio
def have_stock(stock) -> bool:
    try:
        # Check if the stock is in the portfolio
        return any(position.symbol == stock for position in positions)
    except Exception as e:
        print(f"Error checking stock: {e}")
        return False

#return whether or not there is a high volumne

def check_volume(stock_data) -> bool:
    latest_volume = stock_data['Volume'].iloc[-1]  
    high_volume_threshold = 10000000  # Example threshold for high volume
    return latest_volume > high_volume_threshold

def signal_generator(stock) -> int:
    try:
        # Try downloading data with 1 year period, fallback to 3 months if it fails
        stock_data = yf.download(stock, period='1y', interval='1d')
        
        if stock_data.empty:
            print(f"No data available for {stock}, skipping...")
            return 0
        
    except Exception as e:
        print(f"Error downloading {stock}: {e}. Trying with a shorter period.")
        
        # Try a shorter period (e.g., 3 months)
        try:
            stock_data = yf.download(stock, period='3mo', interval='1d')
            if stock_data.empty:
                print(f"No data available for {stock} in the 3-month period, skipping...")
                return -2
        except Exception as e:
            print(f"Error downloading {stock} with shorter period: {e}")
            return -2
    
    # Calculate the moving averages, ensuring enough data is available
    if len(stock_data) < 200:
        print(f"Not enough data for {stock} to calculate 200-day moving average.")
        return 0  # Not enough data for 200-day MA calculation
    
    stock_data['50_day_MA'] = stock_data['Close'].rolling(window=50).mean()
    stock_data['200_day_MA'] = stock_data['Close'].rolling(window=200).mean()
    
    last_row = stock_data.tail(1)
    
    # Check if last row is empty or missing moving averages
    if last_row.empty or pd.isna(last_row['50_day_MA'].iloc[0]) or pd.isna(last_row['200_day_MA'].iloc[0]):
        print(f"Error calculating moving averages for {stock}, skipping...")
        return 0
    
    move_avg_50 = last_row["50_day_MA"].iloc[0]
    move_avg_200 = last_row["200_day_MA"].iloc[0]

    
    if move_avg_50 > move_avg_200 and not have_stock(stock) and check_volume(stock_data):
        return 1  # Buy signal
    elif move_avg_50 < move_avg_200 and have_stock(stock) and check_volume(stock_data):
        return -1  # Sell signal
    elif(have_stock(stock)):
        return 0  # Hold signal
    else:
        return -2


for stock in sp500_stocks:
    signal = signal_generator(stock)
    if signal == 1:
        print(f"Buying {stock}")
    elif signal == -1:
        print(f"Selling {stock}")
    elif signal == 0:
        print(f"Holding {stock}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Buying GOOGL
Holding GOOG


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AMTM']: YFInvalidPeriodError("%ticker%: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']")


Buying AMZN
Buying AMCR
No data available for AMTM, skipping...
Holding AMTM


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

Buying WMT



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Buying WBD



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******